# RareCandy Evaluation and Deployment Notebook

This notebook loads RareCandy exports, computes metrics, runs bootstrap confidence intervals, compares against a baseline, and emits a deployment recommendation (`full`, `canary`, or `no-go`). It also writes CI artifacts: `metrics.json`, plot files, and `manifest.json`.

In [ ]:
from __future__ import annotations

from datetime import datetime, timezone
from pathlib import Path
import json

import numpy as np
import pandas as pd

from metrics import (
    DEFAULT_BARS_PER_YEAR,
    bootstrap_confidence_interval,
    compare_to_baseline,
    compute_equity_from_positions,
    compute_metrics,
    compute_trade_returns,
    deployment_recommendation,
)

try:
    import matplotlib.pyplot as plt
    HAS_MPL = True
except Exception:
    plt = None
    HAS_MPL = False

BASE_DIR = Path.cwd()
EXPORTS_DIR = BASE_DIR / 'exports'
ARTIFACT_DIR = BASE_DIR / 'analysis' / 'artifacts' / 'latest'
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

print('Base directory:', BASE_DIR)
print('Exports directory:', EXPORTS_DIR)
print('Artifact directory:', ARTIFACT_DIR)
print('Matplotlib available:', HAS_MPL)


In [ ]:
REQUIRED_COLUMNS = [
    'timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'position', 'signal'
]

def load_export_pair(stem: str, exports_dir: Path = EXPORTS_DIR) -> pd.DataFrame:
    csv_path = exports_dir / f'{stem}.csv'
    pq_path = exports_dir / f'{stem}.parquet'

    if not csv_path.exists() and not pq_path.exists():
        raise FileNotFoundError(f'Missing both CSV and Parquet for {stem}')

    if pq_path.exists():
        df = pd.read_parquet(pq_path)
    else:
        df = pd.read_csv(csv_path)

    if csv_path.exists() and pq_path.exists():
        csv_rows = len(pd.read_csv(csv_path))
        pq_rows = len(pd.read_parquet(pq_path))
        if csv_rows != pq_rows:
            raise ValueError(f'Row mismatch for {stem}: CSV={csv_rows}, PARQUET={pq_rows}')

    return df


def ensure_schema(df: pd.DataFrame, name: str) -> None:
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f'{name} missing required columns: {missing}')


def bootstrap_sharpe_ci(returns: pd.Series, *, n_boot: int = 2000, ci: float = 0.95, seed: int = 42) -> dict:
    arr = returns.dropna().to_numpy(dtype=float)
    arr = arr[np.isfinite(arr)]
    if arr.size < 2:
        return {'mean': 0.0, 'ci_lower': 0.0, 'ci_upper': 0.0, 'confidence': ci, 'n': int(arr.size), 'n_boot': n_boot}

    rng = np.random.default_rng(seed)
    boots = np.empty(n_boot, dtype=float)
    for i in range(n_boot):
        sample = rng.choice(arr, size=arr.size, replace=True)
        sd = float(np.std(sample, ddof=1))
        boots[i] = float((np.mean(sample) / sd) * np.sqrt(DEFAULT_BARS_PER_YEAR)) if sd > 1e-12 else 0.0

    alpha = (1.0 - ci) / 2.0
    return {
        'mean': float(np.mean(boots)),
        'ci_lower': float(np.quantile(boots, alpha)),
        'ci_upper': float(np.quantile(boots, 1.0 - alpha)),
        'confidence': float(ci),
        'n': int(arr.size),
        'n_boot': int(n_boot),
    }


def _write_svg(path: Path, width: int, height: int, body: str) -> None:
    svg = f"<svg xmlns='http://www.w3.org/2000/svg' width='{width}' height='{height}' viewBox='0 0 {width} {height}'>\n"
    svg += "<rect width='100%' height='100%' fill='white'/>\n" + body + "\n</svg>"
    path.write_text(svg, encoding='utf-8')


def save_line_svg(path: Path, series_map: dict[str, pd.Series], title: str, width: int = 1000, height: int = 400) -> None:
    cleaned = {k: pd.Series(v).dropna().reset_index(drop=True).astype(float) for k, v in series_map.items()}
    max_len = max(len(v) for v in cleaned.values())
    all_vals = np.concatenate([v.to_numpy() for v in cleaned.values() if len(v) > 0])
    y_min = float(np.min(all_vals))
    y_max = float(np.max(all_vals))
    y_rng = (y_max - y_min) if y_max > y_min else 1.0

    margin = 30
    plot_w = width - margin * 2
    plot_h = height - margin * 2

    colors = ['#2563eb', '#dc2626', '#16a34a', '#9333ea', '#ea580c']
    body = [f"<text x='{width//2}' y='20' text-anchor='middle' font-size='14' fill='#222'>{title}</text>"]
    body.append(f"<line x1='{margin}' y1='{height-margin}' x2='{width-margin}' y2='{height-margin}' stroke='#888' stroke-width='1'/>")
    body.append(f"<line x1='{margin}' y1='{margin}' x2='{margin}' y2='{height-margin}' stroke='#888' stroke-width='1'/>")

    for idx, (name, vals) in enumerate(cleaned.items()):
        if len(vals) < 2:
            continue
        pts = []
        for i, val in enumerate(vals):
            x = margin + (i / max(1, max_len - 1)) * plot_w
            y = margin + (1.0 - ((float(val) - y_min) / y_rng)) * plot_h
            pts.append(f"{x:.2f},{y:.2f}")
        col = colors[idx % len(colors)]
        body.append(f"<polyline fill='none' stroke='{col}' stroke-width='2' points='{' '.join(pts)}'/>")
        ly = 40 + idx * 18
        body.append(f"<rect x='{width-180}' y='{ly-12}' width='10' height='10' fill='{col}'/>")
        body.append(f"<text x='{width-165}' y='{ly-3}' font-size='12' fill='#222'>{name}</text>")

    _write_svg(path, width, height, '\n'.join(body))


def save_hist_svg(path: Path, series_map: dict[str, pd.Series], title: str, bins: int = 50, width: int = 1000, height: int = 400) -> None:
    colors = ['#2563eb', '#dc2626']
    margin = 30
    plot_w = width - margin * 2
    plot_h = height - margin * 2

    all_vals = np.concatenate([pd.Series(v).dropna().to_numpy(dtype=float) for v in series_map.values()])
    x_min = float(np.min(all_vals))
    x_max = float(np.max(all_vals))
    if x_max <= x_min:
        x_max = x_min + 1e-6

    body = [f"<text x='{width//2}' y='20' text-anchor='middle' font-size='14' fill='#222'>{title}</text>"]
    body.append(f"<line x1='{margin}' y1='{height-margin}' x2='{width-margin}' y2='{height-margin}' stroke='#888' stroke-width='1'/>")
    body.append(f"<line x1='{margin}' y1='{margin}' x2='{margin}' y2='{height-margin}' stroke='#888' stroke-width='1'/>")

    hist_data = []
    max_count = 1
    for name, series in series_map.items():
        counts, edges = np.histogram(pd.Series(series).dropna().to_numpy(dtype=float), bins=bins, range=(x_min, x_max))
        max_count = max(max_count, int(np.max(counts)))
        hist_data.append((name, counts, edges))

    for idx, (name, counts, edges) in enumerate(hist_data):
        col = colors[idx % len(colors)]
        for b in range(len(counts)):
            left = margin + (edges[b] - x_min) / (x_max - x_min) * plot_w
            right = margin + (edges[b + 1] - x_min) / (x_max - x_min) * plot_w
            h = (counts[b] / max_count) * plot_h
            y = height - margin - h
            body.append(f"<rect x='{left:.2f}' y='{y:.2f}' width='{max(1.0, right-left-0.5):.2f}' height='{h:.2f}' fill='{col}' opacity='0.35'/>")
        ly = 40 + idx * 18
        body.append(f"<rect x='{width-180}' y='{ly-12}' width='10' height='10' fill='{col}'/>")
        body.append(f"<text x='{width-165}' y='{ly-3}' font-size='12' fill='#222'>{name}</text>")

    _write_svg(path, width, height, '\n'.join(body))


In [ ]:
strategy_df = load_export_pair('rarecandy_export')
ensure_schema(strategy_df, 'strategy')

try:
    baseline_df = load_export_pair('baseline_export')
except FileNotFoundError:
    baseline_df = strategy_df.copy()
    baseline_df['position'] = 1.0
    baseline_df['signal'] = 'BUY_HOLD'

ensure_schema(baseline_df, 'baseline')

if 'equity' not in strategy_df.columns:
    strategy_df['equity'] = compute_equity_from_positions(strategy_df)
if 'equity' not in baseline_df.columns:
    baseline_df['equity'] = compute_equity_from_positions(baseline_df, cost_per_side=0.0)

if 'trade_return' not in strategy_df.columns:
    strategy_df['trade_return'] = np.nan
if 'trade_return' not in baseline_df.columns:
    baseline_df['trade_return'] = np.nan

print('Strategy rows:', len(strategy_df))
print('Baseline rows:', len(baseline_df))
display(strategy_df.head(3))


In [ ]:
strategy_metrics = compute_metrics(strategy_df)
baseline_metrics = compute_metrics(baseline_df)
comparison = compare_to_baseline(strategy_metrics, baseline_metrics)

strategy_bar_returns = strategy_df['equity'].pct_change().dropna()
strategy_trade_returns = strategy_df['trade_return'].dropna()
if strategy_trade_returns.empty:
    strategy_trade_returns = compute_trade_returns(strategy_df)

bar_mean_ci = bootstrap_confidence_interval(strategy_bar_returns, n_boot=2000, ci=0.95, seed=42)
trade_mean_ci = bootstrap_confidence_interval(strategy_trade_returns, n_boot=1000, ci=0.95, seed=42) if len(strategy_trade_returns) > 1 else {
    'mean': float(strategy_trade_returns.mean()) if len(strategy_trade_returns) else 0.0,
    'ci_lower': 0.0,
    'ci_upper': 0.0,
    'confidence': 0.95,
    'n': int(len(strategy_trade_returns)),
    'n_boot': 0,
}
sharpe_ci = bootstrap_sharpe_ci(strategy_bar_returns, n_boot=2000, ci=0.95, seed=42)

recommendation = deployment_recommendation(
    strategy_metrics=strategy_metrics,
    baseline_metrics=baseline_metrics,
    sharpe_ci=sharpe_ci,
)

summary_df = pd.DataFrame([
    {'model': 'strategy', **strategy_metrics},
    {'model': 'baseline', **baseline_metrics},
])

display(summary_df[['model', 'total_return_pct', 'sharpe', 'max_drawdown_pct', 'profit_factor', 'num_trades']])
print('Comparison deltas:', comparison)
print('Sharpe CI:', sharpe_ci)
print('Recommendation:', recommendation['mode'], '-', recommendation['summary'])


In [ ]:
strategy_eq = strategy_df['equity']
baseline_eq = baseline_df['equity']
strategy_dd = strategy_eq / strategy_eq.cummax() - 1.0
baseline_dd = baseline_eq / baseline_eq.cummax() - 1.0

plot_paths = []

if HAS_MPL:
    plt.style.use('seaborn-v0_8-darkgrid')

    eq_plot_path = ARTIFACT_DIR / 'equity_curve.png'
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(strategy_eq.values, label='Strategy', linewidth=2)
    ax.plot(baseline_eq.values, label='Baseline', linewidth=2, alpha=0.8)
    ax.set_title('Equity Curve')
    ax.set_xlabel('Bar')
    ax.set_ylabel('Equity')
    ax.legend()
    fig.tight_layout()
    fig.savefig(eq_plot_path, dpi=160)
    plt.close(fig)
    plot_paths.append(eq_plot_path)

    dd_plot_path = ARTIFACT_DIR / 'drawdown_curve.png'
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(strategy_dd.values, label='Strategy DD', linewidth=2)
    ax.plot(baseline_dd.values, label='Baseline DD', linewidth=2, alpha=0.8)
    ax.set_title('Drawdown Curve')
    ax.set_xlabel('Bar')
    ax.set_ylabel('Drawdown')
    ax.legend()
    fig.tight_layout()
    fig.savefig(dd_plot_path, dpi=160)
    plt.close(fig)
    plot_paths.append(dd_plot_path)

    hist_plot_path = ARTIFACT_DIR / 'returns_hist.png'
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.hist(strategy_bar_returns.values, bins=60, alpha=0.7, label='Strategy')
    ax.hist(baseline_eq.pct_change().dropna().values, bins=60, alpha=0.6, label='Baseline')
    ax.set_title('Bar Return Distribution')
    ax.set_xlabel('Return')
    ax.set_ylabel('Count')
    ax.legend()
    fig.tight_layout()
    fig.savefig(hist_plot_path, dpi=160)
    plt.close(fig)
    plot_paths.append(hist_plot_path)
else:
    eq_plot_path = ARTIFACT_DIR / 'equity_curve.svg'
    dd_plot_path = ARTIFACT_DIR / 'drawdown_curve.svg'
    hist_plot_path = ARTIFACT_DIR / 'returns_hist.svg'

    save_line_svg(eq_plot_path, {'Strategy': strategy_eq, 'Baseline': baseline_eq}, 'Equity Curve')
    save_line_svg(dd_plot_path, {'Strategy DD': strategy_dd, 'Baseline DD': baseline_dd}, 'Drawdown Curve')
    save_hist_svg(hist_plot_path, {'Strategy': strategy_bar_returns, 'Baseline': baseline_eq.pct_change().dropna()}, 'Bar Return Distribution')
    plot_paths.extend([eq_plot_path, dd_plot_path, hist_plot_path])

print('Saved plots:')
for p in plot_paths:
    print('-', p)


In [ ]:
metrics_payload = {
    'generated_at_utc': datetime.now(timezone.utc).isoformat(),
    'strategy_metrics': strategy_metrics,
    'baseline_metrics': baseline_metrics,
    'comparison': comparison,
    'bar_mean_ci': bar_mean_ci,
    'trade_mean_ci': trade_mean_ci,
    'sharpe_ci': sharpe_ci,
    'deployment_recommendation': recommendation,
}

metrics_path = ARTIFACT_DIR / 'metrics.json'
manifest_path = ARTIFACT_DIR / 'manifest.json'
recommendation_path = ARTIFACT_DIR / 'deployment_recommendation.txt'

metrics_path.write_text(json.dumps(metrics_payload, indent=2), encoding='utf-8')
recommendation_path.write_text(recommendation['mode'] + '\n', encoding='utf-8')

manifest = {
    'generated_at_utc': datetime.now(timezone.utc).isoformat(),
    'inputs': {
        'strategy_csv': str((EXPORTS_DIR / 'rarecandy_export.csv').resolve()),
        'strategy_parquet': str((EXPORTS_DIR / 'rarecandy_export.parquet').resolve()),
        'baseline_csv': str((EXPORTS_DIR / 'baseline_export.csv').resolve()),
        'baseline_parquet': str((EXPORTS_DIR / 'baseline_export.parquet').resolve()),
    },
    'artifacts': [
        str(metrics_path.resolve()),
        *[str(p.resolve()) for p in plot_paths],
        str(recommendation_path.resolve()),
    ],
    'deployment_mode': recommendation['mode'],
}
manifest_path.write_text(json.dumps(manifest, indent=2), encoding='utf-8')

print('Saved artifact bundle:')
print('-', metrics_path)
print('-', manifest_path)
print('-', recommendation_path)
print('Deployment recommendation:', recommendation['mode'])
